# Reinforcement Learning

## Minihack

### Install NLE

In [ ]:
# !apt update -qq && apt install -qq -y flex bison libbz2-dev libglib2.0-0 libsm6 libxext6 cmake 
# %pip install -U --quiet git+https://github.com/facebookresearch/nle.git@main

### Install Minihack

In [ ]:
# %pip install -U --quiet git+https://github.com/facebookresearch/minihack.git@main

### Install RLlib

In [ ]:
%pip install -U --quiet ray[rllib] ray[tune] ray[default]

### Installs

In [ ]:
%pip install -U --quiet comet_ml hydra-core pipdeptree wandb opencv-python

### Versions

In [1]:
!python --version
!pipdeptree -r --packages pip,gym,nle,minihack,ray,wandb

Python 3.10.6
gym==0.23.0
  - minihack==0.1.3+2f022b0 [requires: gym>=0.15,<=0.23]
  - nle==0.8.1+68b9362 [requires: gym>=0.15,<=0.23]
    - minihack==0.1.3+2f022b0 [requires: nle>=0.8.1]
pip==22.3.1
ray==2.0.1
wandb==0.13.5


### Imports

In [2]:
import random
import gym
import nle
import minihack
import ray

import numpy as np
import cv2
from collections import OrderedDict

In [3]:
ray.init(num_gpus=1)

2022-11-08 17:28:57,445	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


### Custom

In [6]:
from gym.spaces import Box
from minihack.envs.skills_quest import MiniHackQuestHard
from ray.tune.registry import register_env


class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class CustomEnv(MiniHackQuestHard):
    def __init__(self, config):
        # Hack to resolve error "'CustomEnv' object has no attribute 'env'"
        self.env = dotdict({"_vardir": "/tmp/run"})

        config = dotdict(config)

        self._obs_keys = config.obs_keys.split(",")
        super().__init__(observation_keys=self._obs_keys)

        self.shape = dotdict(config.input_shape)
        self.observation_space["pixel"] = Box(
            0, 255, (self.shape.height, self.shape.width, 3), np.uint8
        )

        print(self.observation_space)

    def _resize_frame(self, frame):
        return cv2.resize(
            frame,
            dsize=(self.shape.width, self.shape.height),
            interpolation=cv2.INTER_LINEAR,
        )

    def _process_obs(self, obs):
        return OrderedDict(
            {
                key: self._resize_frame(obs[key]) if key == "pixel" else obs[key]
                for key in self._obs_keys
            }
        )

    def reset(self):
        return self._process_obs(super().reset())

    def step(self, action):
        obs, reward, done, info = super().step(action)
        return self._process_obs(obs), reward, done, info


register_env("MiniHack-D3QN-v0", CustomEnv)


### Train

In [5]:
from hydra import initialize, compose
from omegaconf import OmegaConf

import ray
from ray import tune
from ray.air.callbacks.wandb import WandbLoggerCallback

with initialize(version_base=None, config_path="."):
    cfg = compose(config_name="config.yaml")

dqn_cfg = OmegaConf.to_object(cfg.get("minihack-d3qn", {}))

wandb_cfg = dqn_cfg.get("logger_config", {}).get("wandb", {})

callbacks = [
    WandbLoggerCallback(
        project=wandb_cfg["project"],
        group=wandb_cfg["group"],
        api_key_file=wandb_cfg["api_key_file"],
    )
]

analysis = tune.run(
    "DQN",
    callbacks=callbacks,
    config=dqn_cfg,
    stop={"training_iteration": 10},
    local_dir="./results",
    log_to_file=True,
)


Trial name,status,loc,gamma,hiddens,lr,target_network_up...
DQN_MiniHack-D3QN-v0_ab7af_00011,RUNNING,,0.999,[512],0.0001,10000
DQN_MiniHack-D3QN-v0_ab7af_00012,PENDING,,0.99,[256],0.001,10000
DQN_MiniHack-D3QN-v0_ab7af_00013,PENDING,,0.999,[256],0.001,10000
DQN_MiniHack-D3QN-v0_ab7af_00014,PENDING,,0.99,[512],0.001,10000
DQN_MiniHack-D3QN-v0_ab7af_00015,PENDING,,0.999,[512],0.001,10000
DQN_MiniHack-D3QN-v0_ab7af_00016,PENDING,,0.99,[256],0.0001,50000
DQN_MiniHack-D3QN-v0_ab7af_00017,PENDING,,0.999,[256],0.0001,50000
DQN_MiniHack-D3QN-v0_ab7af_00018,PENDING,,0.99,[512],0.0001,50000
DQN_MiniHack-D3QN-v0_ab7af_00019,PENDING,,0.999,[512],0.0001,50000
DQN_MiniHack-D3QN-v0_ab7af_00020,PENDING,,0.99,[256],0.001,50000


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mamello-justice (coms-4061a-team). Use `wandb login --relogin` to force relogin


wandb: ERROR Failed to sample metric: Not Supported
(DQN pid=2041) 2022-11-08 17:47:45,745	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.algorithms.dqn.dqn.DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.algorithms.dqn.dqn.DQNConfig(...)` instead. This will raise an error in the future!
(DQN pid=2041) 2022-11-08 17:47:45,745	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
(DQN pid=2041) 2022-11-08 17:47:45,746	INFO simple_q.py:293 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQN pid=2041) 2022-11-08 17:47:45,748	INFO algorithm.py:355 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQN pid=2041) 2022-11-08 17:47:45,817	WARNING env.py:142 -- Y

2022-11-08 17:47:51,386	ERROR trial_runner.py:987 -- Trial DQN_MiniHack-D3QN-v0_ab7af_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 996, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/_private/worker.py", line 2282, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=2041, ip=172.21.111.144, repr=DQN)
  File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 312, in __init__
    supe

(DQN pid=2041) 2022-11-08 17:47:51,360	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=2041, ip=172.21.111.144, repr=DQN)
(DQN pid=2041)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 312, in __init__
(DQN pid=2041)     super().__init__(config=config, logger_creator=logger_creator, **kwargs)
(DQN pid=2041)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 159, in __init__
(DQN pid=2041)     self.setup(copy.deepcopy(self.config))
(DQN pid=2041)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 422, in setup
(DQN pid=2041)     self.workers = WorkerSet(
(DQN pid=2041)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 1

...

wandb: ERROR Failed to sample metric: Not Supported
(DQN pid=3870) 2022-11-08 17:50:46,599	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.algorithms.dqn.dqn.DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.algorithms.dqn.dqn.DQNConfig(...)` instead. This will raise an error in the future!
(DQN pid=3870) 2022-11-08 17:50:46,599	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
(DQN pid=3870) 2022-11-08 17:50:46,600	INFO simple_q.py:293 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQN pid=3870) 2022-11-08 17:50:46,602	INFO algorithm.py:355 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQN pid=3870) 2022-11-08 17:50:46,661	WARNING env.py:142 -- Y

Result for DQN_MiniHack-D3QN-v0_ab7af_00010:
  trial_id: ab7af_00010
  


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mamello-justice (coms-4061a-team). Use `wandb login --relogin` to force relogin


wandb: ERROR Failed to sample metric: Not Supported
2022-11-08 17:51:06,014	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(DQN pid=4073) 2022-11-08 17:51:07,727	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.algorithms.dqn.dqn.DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.algorithms.dqn.dqn.DQNConfig(...)` instead. This will raise an error in the future!
(DQN pid=4073) 2022-11-08 17:51:07,727	WARNING deprecation.py:47 -- DeprecationWarning: `config['multiagent']['replay_mode']` has been deprecated. config['replay_buffer_config']['replay_mode'] This will raise an error in the future!
(DQN pid=4073) 2022-11-08 17:51:07,727	INFO simple_q.py:293 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQ

(DQN pid=4073) 2022-11-08 17:51:12,448	ERROR worker.py:756 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=4073, ip=172.21.111.144, repr=DQN)
(DQN pid=4073)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 312, in __init__
(DQN pid=4073)     super().__init__(config=config, logger_creator=logger_creator, **kwargs)
(DQN pid=4073)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 159, in __init__
(DQN pid=4073)     self.setup(copy.deepcopy(self.config))
(DQN pid=4073)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 422, in setup
(DQN pid=4073)     self.workers = WorkerSet(
(DQN pid=4073)   File "/home/develop/anaconda3/envs/minihack_rl/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 1

In [ ]:
ray.shutdown()